<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/product_id_%EB%B3%84%EB%A1%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%82%98%EB%88%A0%EC%84%9C(%EB%B6%80%EB%B6%84_kn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')
test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

Mounted at /content/drive


In [ ]:
qual_col = ['LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_df[i])
    train_df[i] = le.transform(train_df[i])
    
    for label in np.unique(test_df[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
    
print('Done.')


Done.


In [ ]:

train_a = train_df[train_df['PRODUCT_CODE']=='A_31']
train_a_y = train_a['Y_Class']
train_a_quality = train_a['Y_Quality']
train_a_x = train_a.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality', 'PRODUCT_CODE'])

train_to = train_df[(train_df['PRODUCT_CODE']=='T_31') | (train_df['PRODUCT_CODE']=='O_31')]
train_to_y = train_to['Y_Class']
train_to_quality = train_to['Y_Quality']
train_to_x = train_to.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality', 'PRODUCT_CODE'])

#test_df = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
#test_a = test_df[test_df['PRODUCT_CODE']=='A_31']
#test_to = test_df[(train_df['PRODUCT_CODE']=='T_31') | (train_df['PRODUCT_CODE']=='O_31')]

print(train_to_x.shape)
print(train_a_x.shape)

(349, 2876)
(249, 2876)


In [ ]:
#그래서 하나의 세트에서 모두 결측치인 열은 두 세트에서 모두 삭제 
invalid_a=[]
invalid_to=[]
ori=['Index', 'LINE', 'PRODUCT_CODE']
for col in train_a_x.columns:
  if(col!='LINE'):
    ori+=[col]
  if(train_a_x[col].isnull().all()):
    invalid_a += [col]
for col in train_to_x.columns:
  if(train_to_x[col].isnull().all()):
    invalid_to += [col]

train_a_x = train_a_x.drop(columns=invalid_a)
train_to_x = train_to_x.drop(columns=invalid_to)
#test = test.drop(columns=invalid_a)
print(train_a_x.shape)
print(train_to_x.shape)
#print(test.shape)



(249, 2117)
(349, 678)


In [ ]:
RF_a =  XGBRegressor(objective='reg:squarederror')
RF_a.fit(train_a_x, train_a_quality)

RF_to =  XGBRegressor(objective='reg:squarederror')
RF_to.fit(train_to_x, train_to_quality)

print('Done.')

Done.


In [ ]:
over_a = train_a[train_a['Y_Class'] == 2]['Y_Quality']
normal_a = train_a[train_a['Y_Class'] == 1]['Y_Quality']
under_a = train_a[train_a['Y_Class'] == 0]['Y_Quality']

over_to = train_to[train_to['Y_Class'] == 2]['Y_Quality']
normal_to = train_to[train_to['Y_Class'] == 1]['Y_Quality']
under_to = train_to[train_to['Y_Class'] == 0]['Y_Quality']

from sklearn.neighbors import KNeighborsClassifier
kn_a = KNeighborsClassifier(n_neighbors=5)
train_a_quality = train_a_quality.values.reshape(-1,1)
kn_a.fit(train_a_quality, train_a_y)

kn_to = KNeighborsClassifier(n_neighbors=5)
train_to_quality = train_to_quality.values.reshape(-1,1)
kn_to.fit(train_to_quality, train_to_y)

KNeighborsClassifier()

In [ ]:



def classify_a(y_quality):
    if normal_a.min() <= y_quality <= normal_a.max():
        return 1
    elif y_quality <= under_a.max():
        return 0
    elif over_a.min() <= y_quality:
        return 2
    else:
        #return -1
        # 커트라인 확정이 안되는 경우
        temp=kn_a.predict([y_quality])[0]
        return temp

def classify_to(y_quality): 
    if normal_to.min() <= y_quality <= normal_to.max():
        return 1
    elif y_quality <= under_to.max():
        return 0
    elif over_to.min() <= y_quality:
        return 2
    else:
        #return -1
        # 커트라인 확정이 안되는 경우
        temp= kn_to.predict([y_quality])[0]  
        return temp    

In [ ]:
preds=[]
for row in test.itertuples():
  tmp = pd.DataFrame([row], columns=ori)
  if(tmp.loc[0, 'PRODUCT_CODE']=='A_31'):
    tmp = tmp.drop(columns=invalid_a)
    tmp = tmp.drop(columns='PRODUCT_CODE')
    tmp = tmp.drop(columns='Index')
    mid = RF_a.predict(tmp)
    result = classify_a(mid)
  else:
    tmp = tmp.drop(columns=invalid_to)
    tmp = tmp.drop(columns='PRODUCT_CODE')
    tmp = tmp.drop(columns='Index')
    mid = RF_to.predict(tmp)
    result = classify_to(mid)
  preds+=[result]


#preds = RF.predict(test_a)
print('Done.')



Done.


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)